<light>First some imports we need</light>

In [ ]:
import operator, math, random
import numpy
from deap import base, creator, tools, gp, algorithms
from helper import plot, plot_tree

<h2>1. Defining the PrimitiveSet Object</h2>
<p>
    In this first step we define the PrimitiveSet <code>pset</code>. The <code>pset</code> contains  the base functions, constants and the definition of input parameters. In following steps trees are     build out of these components.
</p>
</br>
<p>
    Generate a PrimitiveSetType Object with two input parameters (first one is a boolean and second one is a float). The return value of the result (the root node in the tree) should be a float value.
</p>

In [ ]:
pset = gp.PrimitiveSetTyped("main", [bool, float], float)

Define a custom function

In [ ]:
def if_then_else(input, output1, output2):
    return output1 if input else output2

Define primitive functions for the nodes. The first parameter defines the function, the second the types of the input parameters and the third one the type of the output.

In [ ]:
pset.addPrimitive(operator.xor, [bool, bool], bool)
pset.addPrimitive(operator.mul, [float, float], float)
pset.addPrimitive(if_then_else, [bool, float, float], float)

Secondly define some constants for the terminals (leafs in the tree)

In [ ]:
pset.addTerminal(3.0, float)
pset.addTerminal(1, bool)

Define names for the input parameters

In [ ]:
pset.renameArguments(ARG0="x")
pset.renameArguments(ARG1="y")

<h2>2. Generation of Tree Individuals</h2>
<p>

To generate valid individuals we need to create the <code>Fitness</code> and <code>Individuals</code> classes. We also add a reference to the previously generated PrimitiveSet object.

</p>

In [ ]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin,
               pset=pset)

Register the functions into a toolbox.

In [ ]:
toolbox = base.Toolbox()
toolbox.register("expr", gp.genFull, pset=pset, min_=1, max_=3)
toolbox.register("individual", tools.initIterate, creator.Individual,
                 toolbox.expr)

<h2>3. Evaluation of trees</h2>

In [ ]:
expr = gp.genFull(pset, min_=1, max_=3)
tree = gp.PrimitiveTree(expr)
str(tree)

In [ ]:
function = gp.compile(tree, pset)
function(1, 2)

<h2>4. Plotting a tree</h2>

In [ ]:
nodes, edges, labels = gp.graph(expr)
plot_tree(nodes, edges, labels)

<img src="./tree.png"/>